# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234165447172                   -0.50    6.0
  2   -7.249725939868       -1.81       -1.40    1.0
  3   -7.251273784797       -2.81       -1.97    4.0
  4   -7.251273691677   +   -7.03       -2.17    4.0
  5   -7.251333854499       -4.22       -2.74    4.0
  6   -7.251336554804       -5.57       -2.99    2.0
  7   -7.251338740184       -5.66       -3.77    2.0
  8   -7.251338789553       -7.31       -4.10    2.0
  9   -7.251338798307       -8.06       -4.84    2.0
 10   -7.251338798680       -9.43       -5.31    3.0
 11   -7.251338798695      -10.84       -5.56    3.0
 12   -7.251338798704      -11.04       -6.13    2.0
 13   -7.251338798705      -12.17       -7.05    3.0
 14   -7.251338798705      -13.80       -7.15    4.0
 15   -7.251338798705      -14.21       -7.57    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06064078200585771
[ Info: Arnoldi iteration step 2: normres = 0.4398688717590279
[ Info: Arnoldi iteration step 3: normres = 0.748885025887573
[ Info: Arnoldi iteration step 4: normres = 0.47830535184091827
[ Info: Arnoldi iteration step 5: normres = 0.47080131203904296
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.52e-02, 6.52e-02, 3.23e-01, 3.22e-01, 1.63e-02)
[ Info: Arnoldi iteration step 6: normres = 0.26089883422860577
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.11e-02, 7.91e-02, 2.25e-01, 6.63e-02, 5.56e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06987886022720839
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.03e-04, 6.12e-03, 1.28e-02, 1.96e-02, 5.70e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09996993935669804
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.95e